In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import scipy as sp
import time

In [2]:
def initialize_wf(N,N_pt):
    coords=np.sort(random.sample(range(N),N_pt))
    n_occ=np.zeros((N,))        
    n_occ[coords]=np.ones((N_pt,))        
    n_pos = np.zeros((N,),dtype=int)        
    n_pos[coords] = np.arange(1,N_pt+1)
    
    return n_occ, n_pos, coords

def initialize_wf_region(N, N_pt, inds_A, N_pt_A):
    
    inds_A = inds_A.tolist()
    inds_outsideA = np.arange(N)
    inds_outsideA = np.delete(inds_outsideA, inds_A).tolist()
    N_pt_remainder = N_pt-N_pt_A

    assert N_pt_A <= len(inds_A), 'N_pt_A > subsystem A size'
    assert N_pt_remainder <= N-len(inds_A), 'N_pt_B > subsystem B size'

    coords = [0]*N_pt       
    coords[:N_pt_A] = np.sort(random.sample(inds_A, N_pt_A))
    coords[N_pt_A:] = np.sort(random.sample(inds_outsideA, N_pt_remainder))
#     print(coords)
    n_occ = np.zeros((N,))        
    n_occ[coords] = np.ones((N_pt,))    
    n_pos = np.zeros((N,),dtype=int)        
    n_pos[coords] = np.arange(1,N_pt+1)
    
    return n_occ, n_pos, coords[:N_pt]

In [9]:
def wf_gen(N,N_pt,BC,t1,t2):
    h1=np.ones(N-1)
    h1[0::2]=0
    h2=np.ones(N-1)
    h2[1::2]=0
    hop= np.diag(t1*h1+t2*h2+0j,1)
    hop[N-1,0]= t1*BC
    H_t= -(hop+ np.matrix(hop).H)/2 
    energies, evecs= np.linalg.eigh(H_t)
    return evecs[:,:N_pt]

def WF_MAT(wf_gen, config):
    n_e = len(config)
    wf = np.zeros((n_e,n_e),dtype = "complex")
    wf = np.transpose(wf_gen[config,:])
     
    return wf

def exact_renyi_calc(r,GA,epsilon=1e-9):
    chi0, _ =np.linalg.eigh(GA)
    chi1=chi0[np.nonzero(np.abs(chi0)>epsilon)]
    chi2=chi1[np.nonzero(np.abs(chi1-1)>epsilon)] #/np.sum(chi1[np.nonzero(np.abs(chi1-1)>epsilon)])
    return np.sum(np.log((1-chi2)**r+chi2**r))/(1-r) #np.sum(np.log((1-chi2)**r+chi2**r))/(1-r) #Twice as large....
    #return np.log(np.sum(chi2**r))/(1-r) 

In [27]:
def VMC_equal_number(numconfig,check_step,N,N_pt,wf_gen,wf_r,n_occ_r,coords_r,n_pos_r,inds_A):


    move_attempted = 0
    move_accepted = 0
    
    r = wf_r.shape[2]
    
    wf_inv_r = np.zeros(wf_r.shape,dtype=np.complex128)
    for i_r in range(r):
        wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])
        
    min_state=5
    ratio_0=1.0+0.0j
    
    
    fraction=np.zeros(len(inds_A)+1) #a count for each potential particle number in the region 
    
    # possible steps
    step_abs=np.arange(1,Ns+1) #Ns?
    step_vals=np.sort(np.concatenate((-step_abs,step_abs),axis=0)).tolist()

    for state in range(numconfig+min_state):
        for step in range(0,check_step):               #Samples observables after check_step iterations
        
            for i_r in range(r):
                for moved_elec in range(N_pt):
                    move_attempted=move_attempted+1

                    # random walk of Ns steps left or right
                    stepx=random.sample(step_vals,1)[0]
                    ptcls_x= np.mod( coords_r[moved_elec,i_r]+stepx, N) # new configuration

                    if n_occ_r[ptcls_x,i_r]==1:
                        continue

                    u_0 = np.transpose(wf_gen[ptcls_x,:] - wf_gen[coords_r[moved_elec,i_r],:])
                    #pt_wf_1=np.transpose(wf_gen[ptcls_x,:])

                    rel =  np.dot(wf_gen[ptcls_x,:],wf_inv_r[moved_elec,:,i_r].reshape((N_pt,1)))
                    #rel=np.dot(inv_wf[moved_elec,:],pt_wf_1)

                    alpha=min(1, np.abs(rel)**2)

                    random_num=random.random()

                    if random_num <= alpha:
                        move_accepted += 1

                        #Sherman-Morisson Formula for updating inverse matrix
                        v=np.zeros((N_pt,1))
                        v[moved_elec]=1
                        num = np.dot(np.dot(np.dot(wf_inv_r[:,:,i_r],u_0),v.T),wf_inv_r[:,:,i_r])
                        denom = (1+np.dot(v.T,np.dot(wf_inv_r[:,:,i_r],u_0)))
                        wf_inv_r[:,:,i_r] = wf_inv_r[:,:,i_r] - num/denom 
                        wf_r[:,moved_elec,i_r] = np.reshape(np.transpose(wf_gen[ptcls_x,:]),(N_pt,))

                        delta = np.zeros(N)
                        delta[coords_r[moved_elec,i_r]] = -1
                        delta[ptcls_x]=1

                        n_occ_r[:,i_r] = n_occ_r[:,i_r] + delta
                        n_pos_r[:,i_r] = n_pos_r[:,i_r] + (moved_elec + 1)*delta
                        coords_r[moved_elec,i_r] = ptcls_x 
                        
                x=np.argwhere(n_occ_r[:,i_r]>0)
                assert len(x)== N_pt, 'no of ptcle is %d' % (len(x))

#################################################################                
        if state> (min_state-1):
            pt_num_inside= np.sum(n_occ_r[inds_A,:],axis=0)
            
            if np.max(pt_num_inside)==np.min(pt_num_inside) :
                fraction[int(pt_num_inside[0])] += 1
                
                
        if (state%500) ==0:
            for i_r in range(r):
                wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])

    acc_ratio=move_accepted/move_attempted
    fraction = fraction/(numconfig)        
    
    #print("fraction acceptance rate=", acc_ratio)
    return fraction

In [5]:
def VMC_amplitude_ratio(numconfig,check_step,N,N_pt,wf_gen,wf_r,n_occ_r,n_pos_r,coords_r,inds_A,N_pt_A):

    move_attempted = 0
    move_accepted = 0
    
    r = wf_r.shape[2]
    
    wf_inv_r = np.zeros(wf_r.shape,dtype=np.complex128)
    wf_inds= np.zeros((N_pt_A,r),dtype=int)
    for i_r in range(r):
        wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])
        pt_num_inside = np.argwhere( n_occ_r[inds_A,i_r]>0 )
        pt_num_inside = np.reshape( pt_num_inside, (N_pt_A,)).tolist()
        wf_inds[:,i_r]=( n_pos_r[ inds_A[pt_num_inside], i_r] )-1   #Used to be sorted
    
    wf_swap = np.zeros(wf_r.shape,dtype=np.complex128)
    up_one = (np.arange(r)+1)%r
    for i_r in range(r):
    # r permutation of subsystem indices
        wf_swap[:,:,i_r] = np.copy(wf_r[:,:,i_r])
        wf_swap[:,wf_inds[:,i_r],i_r] = np.copy(wf_r[:,wf_inds[:,up_one[i_r]],up_one[i_r]]) #Shifting configurations with the next wf in sequence

    inside_A = 0 #indicator of location of melectron to be moved
    
    step_abs=np.arange(1,Ns+1) #Ns?
    step_vals=np.sort(np.concatenate((-step_abs,step_abs),axis=0)).tolist()
        
    count=0 # counter for entropy
    min_state=5
    
    ent_ratio=np.zeros(int(numconfig),dtype=np.float64) 
    
    for state in range(numconfig+min_state):
        for step in range(0,check_step):               #Samples observables after check_step iterations
        
            for i_r in range(r):
                for moved_elec in range(N_pt):
                    move_attempted=move_attempted+1
                    
                    stepx=random.sample(step_vals,1)[0]
                    ptcls_x= np.mod( coords_r[moved_elec,i_r]+stepx, N) # new configuration

                    if n_occ_r[ptcls_x,i_r]==1:
                        continue

                    val_orig = np.min( np.abs(inds_A- coords_r[moved_elec,i_r]) )
                    val_dest = np.min( np.abs(inds_A-ptcls_x) )
                    if val_orig==0 and val_dest==0 :
                        inside_A=1
                        val_1 = np.min(np.abs(wf_inds[:,i_r]-moved_elec))
                        ind_1 = np.argmin(np.abs(wf_inds[:,i_r]-moved_elec))
                        assert val_1==0 , 'wrong inds_A for wf'
                    elif val_orig!=0 and val_dest!=0 :
                        inside_A=0
                    else:
                        continue

                    u_0 = np.transpose(wf_gen[ptcls_x,:] - wf_gen[coords_r[moved_elec,i_r],:])
                    #pt_wf_1=np.transpose(wf_gen[ptcls_x,:])

                    rel =  np.dot(wf_gen[ptcls_x,:],wf_inv_r[moved_elec,:,i_r].reshape((N_pt,1)))
                    #rel=np.dot(inv_wf[moved_elec,:],pt_wf_1)

                    alpha=min(1, np.abs(rel)**2)

                    random_num=random.random()

                    if random_num <= alpha:
                        move_accepted += 1

                        #Sherman-Morisson Formula for updating inverse matrix
                        v=np.zeros((N_pt,1))
                        v[moved_elec]=1
                        num = np.dot(np.dot(np.dot(wf_inv_r[:,:,i_r],u_0),v.T),wf_inv_r[:,:,i_r])
                        denom = (1+np.dot(v.T,np.dot(wf_inv_r[:,:,i_r],u_0)))
                        wf_inv_r[:,:,i_r] = wf_inv_r[:,:,i_r] - num/denom 
                        wf_r[:,moved_elec,i_r] = np.reshape(np.transpose(wf_gen[ptcls_x,:]),(N_pt,))
                        
                        if inside_A==1:
                            wf_swap[:,wf_inds[ind_1,up_one[i_r]],up_one[i_r]]= wf_gen[ptcls_x,:] #CHECK!
                            inside_A=0
                        else:
                            wf_swap[:,moved_elec,i_r]= wf_gen[ptcls_x,:]

                        delta = np.zeros(N)
                        delta[coords_r[moved_elec,i_r]] = -1
                        delta[ptcls_x]=1

                        n_occ_r[:,i_r] = n_occ_r[:,i_r] + delta
                        n_pos_r[:,i_r] = n_pos_r[:,i_r] + (moved_elec + 1)*delta
                        coords_r[moved_elec,i_r] = ptcls_x

                x=np.argwhere(n_pos_r[:,i_r]>0)
                assert len(x)== N_pt, 'no of ptcle is %d' % (len(x))
                assert np.sum(n_occ_r[inds_A,i_r])== N_pt_A, 'n_occ_A anc ptcle is %d' % (np.sum(n_occ_r[inds_A,i_r]))
                assert np.sum(n_occ_r[:,i_r])== N_pt, 'n_occ anc ptcle is %d' % (np.sum(n_occ_r[:,i_r]))


# ##############################################################     
            
        if state> (min_state-1):
            
            ratio_r=1.0+0j
            for i_r in range(r):
                ratio_r *= np.linalg.det(wf_swap[:,:,i_r])/np.linalg.det(wf_r[:,:,i_r]) #product of r transposed ratios
            ent_ratio[count] = np.abs(ratio_r)
            count+=1

        
        if (state%500) ==0:
            for i_r in range(r):
                wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])


    acc_ratio=move_accepted/move_attempted
    
    
    #print("Amplitude acceptance rate=", acc_ratio)
    return np.mean(ent_ratio)

In [32]:
def VMC_phase_ratio(numconfig,check_step,N,N_pt,wf_gen,wf_r,n_occ_r,n_pos_r,coords_r,inds_A,N_pt_A):

    move_attempted = 0
    move_accepted = 0
    
    r = wf_r.shape[2]
    up_one = (np.arange(r)+1)%r
    down_one = (np.arange(r)-1)%r
    
    wf_inv_r = np.zeros(wf_r.shape,dtype=np.complex128)
    wf_inds= np.zeros((N_pt_A,r),dtype=int)
    for i_r in range(r):
        wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])
        pt_num_inside = np.argwhere( n_occ_r[inds_A,i_r]>0 )
        pt_num_inside = np.reshape( pt_num_inside, (N_pt_A,)).tolist()
        wf_inds[:,i_r]=( n_pos_r[ inds_A[pt_num_inside], i_r] )-1    #Used to be sorted
    
    wf_swap = np.zeros(wf_r.shape,dtype=np.complex128)
    wf_swap_inv = np.zeros(wf_r.shape,dtype=np.complex128)
    for i_r in range(r):
    # r permutation of subsystem indices
        wf_swap[:,:,i_r] = np.copy(wf_r[:,:,i_r])
        wf_swap[:,wf_inds[:,i_r],i_r] = np.copy(wf_r[:,wf_inds[:,up_one[i_r]],up_one[i_r]]) #Shifting configurations with the next wf in sequence
        wf_swap_inv[:,:,i_r] = np.linalg.inv(wf_swap[:,:,i_r])

    step_abs=np.arange(1,Ns+1) #Ns?
    step_vals=np.sort(np.concatenate((-step_abs,step_abs),axis=0)).tolist()

    
    min_state=5
    
    phi_0 = 1
    for i_r in range(r):
        phi_0*=np.conj(np.linalg.det(wf_r[:,:,i_r]))*np.linalg.det(wf_swap[:,:,i_r])
    phi_0= np.angle(phi_0)
    
    psi = phi_0
    #phi=phi_0
#     phi=np.exp(1j*phi_0)

    ent_ratio=np.zeros(numconfig,dtype=np.complex64)
    ent_ratio_comp=np.zeros(numconfig,dtype=np.complex64)
    
    for state in range(numconfig+min_state):
        for step in range(0,check_step):               #Samples observables after check_step iterations
        
            for i_r in range(r):
                for moved_elec in range(N_pt):
                    move_attempted=move_attempted+1
                    
                    stepx=random.sample(step_vals,1)[0]
                    ptcls_x= np.mod( coords_r[moved_elec,i_r]+stepx, N) # new configuration

                    if n_occ_r[ptcls_x,i_r]==1:
                        continue

                    inside_A=0    
                    val_orig = np.min( np.abs(inds_A- coords_r[moved_elec,i_r]) )
                    val_dest = np.min( np.abs(inds_A-ptcls_x) )
                    if val_orig==0 and val_dest==0 :
                        inside_A=1
                        val_1 = np.min(np.abs(wf_inds[:,i_r]-moved_elec))
                        ind_1 = np.argmin(np.abs(wf_inds[:,i_r]-moved_elec))
                        assert val_1==0 , 'wrong inds_A for wf'
                        rel = np.dot(wf_swap_inv[wf_inds[ind_1,up_one[i_r]],:,up_one[i_r]],np.transpose(wf_gen[ptcls_x,:])) #down_one[i_r]?
                    elif val_orig!=0 and val_dest!=0 :
                        rel = np.dot(wf_swap_inv[moved_elec,:,i_r],np.transpose(wf_gen[ptcls_x,:])) #down_one[i_r]?
                        inside_A=0
                    else:
                        continue
                        
                    u_0 = np.transpose(wf_gen[ptcls_x,:] - wf_gen[coords_r[moved_elec,i_r],:])
                    pt_wf_1=np.transpose(wf_gen[ptcls_x,:])

                    rel =  rel*np.conj(np.dot(wf_inv_r[moved_elec,:,i_r],pt_wf_1))

                    alpha=min(1, np.abs(rel))

                    random_num=random.random()

                    if random_num <= alpha:
                        move_accepted += 1

                        #Sherman-Morisson Formula for updating inverse matrix
                        v=np.zeros((N_pt,1))
                        v[moved_elec]=1
                        num = np.dot(np.dot(np.dot(wf_inv_r[:,:,i_r],u_0),v.T),wf_inv_r[:,:,i_r])
                        denom = (1+np.dot(v.T,np.dot(wf_inv_r[:,:,i_r],u_0)))
                        wf_inv_r[:,:,i_r] = wf_inv_r[:,:,i_r] - num/denom 
                        wf_r[:,moved_elec,i_r] = np.reshape(np.transpose(wf_gen[ptcls_x,:]),(N_pt,))
                        
                        if inside_A==1:
                            u_1=np.reshape(pt_wf_1,(N_pt,1)) - np.reshape(wf_swap[:,wf_inds[ind_1,up_one[i_r]],up_one[i_r]],(N_pt,1))
                            v=np.zeros((N_pt,1))
                            v[wf_inds[ind_1,up_one[i_r]]]=1
                            wf_swap_inv[:,:,up_one[i_r]]=wf_swap_inv[:,:,up_one[i_r]] - np.dot(np.dot(np.dot(wf_swap_inv[:,:,up_one[i_r]],u_1),v.T),wf_swap_inv[:,:,up_one[i_r]]) \
                                 /(1+np.dot(v.T,np.dot(wf_swap_inv[:,:,up_one[i_r]],u_1))) 
                            wf_swap[:,wf_inds[ind_1,up_one[i_r]],up_one[i_r]]= np.reshape(pt_wf_1,(N_pt,))
                            inside_A=0
                        else:
                            u_1=np.reshape(pt_wf_1,(N_pt,1)) - np.reshape(wf_swap[:,moved_elec,i_r],(N_pt,1))
                            wf_swap_inv[:,:,i_r]=wf_swap_inv[:,:,i_r] - np.dot(np.dot(np.dot(wf_swap_inv[:,:,i_r],u_1),v.T),wf_swap_inv[:,:,i_r]) \
                                    /(1+np.dot(v.T,np.dot(wf_swap_inv[:,:,i_r],u_1))) 

                            wf_swap[:,moved_elec,i_r]= np.reshape(pt_wf_1,(N_pt,))
                        
                        delta = np.zeros(N)
                        delta[coords_r[moved_elec,i_r]] = -1
                        delta[ptcls_x]=1

                        n_occ_r[:,i_r] = n_occ_r[:,i_r] + delta
                        n_pos_r[:,i_r] = n_pos_r[:,i_r] + (moved_elec + 1)*delta
                        coords_r[moved_elec,i_r] = ptcls_x
                        
                        #phi = phi + np.angle(rel)
                        psi = psi + np.angle(rel)

                x=np.argwhere(n_pos_r[:,i_r]>0)
                assert len(x)== N_pt, 'no of ptcle is %d' % (len(x))
                assert np.sum(n_occ_r[inds_A,i_r])== N_pt_A, 'n_occ_A anc ptcle is %d' % (np.sum(n_occ_r[inds_A,i_r]))
                assert np.sum(n_occ_r[:,i_r])== N_pt, 'n_occ anc ptcle is %d' % (np.sum(n_occ_r[:,i_r]))
                        



# ##############################################################
        assert n_occ_r[:,i_r].sum()==N_pt, "total particle number changes!!"
    
        if state> (min_state-1):
            #phi = 1
            #for i_r in range(r):
            #    phi*=np.conj(np.linalg.det(wf_r[:,:,i_r]))*np.linalg.det(wf_swap[:,:,i_r])
            #phi= np.angle(phi)
            #ent_ratio[state-min_state] = phi
            ent_ratio[state-min_state] = psi

        
        if (state%500) ==0:
            for i_r in range(r):
                wf_inv_r[:,:,i_r]=np.linalg.inv(wf_r[:,:,i_r])
                wf_swap_inv[:,:,i_r] = np.linalg.inv(wf_swap[:,:,i_r])
        #print("Rel is:", rel)


    acc_ratio=move_accepted/move_attempted

        
    #print("Angle acceptance rate=", acc_ratio)
    return np.mean(np.exp(1j*ent_ratio))
#     return np.mean(ent_ratio)

In [84]:
def Renyi_vmc(numconfig,wf_gen,N, N_pt,Lsub,r):
    
    inds_A= np.arange(0,Lsub)
    
    n_occ_r = np.zeros((N,r))
    n_pos_r = np.zeros((N,r))
    coords_r = np.zeros((N_pt,r),dtype = 'int')
    wf_r = np.zeros((N_pt,N_pt,r),dtype = 'complex')
    cond_r= np.zeros(r)
    for i_r in range(r):
        n_occ_r[:,i_r], n_pos_r[:,i_r], coords_r[:,i_r]=initialize_wf(N, N_pt)
        wf_r[:,:,i_r]=np.transpose(wf_gen[coords_r[:,i_r],:])
        cond_r[i_r]=np.linalg.cond(wf_r[:,:,i_r])
        
    Frac = VMC_equal_number(numconfig,check_step,N,N_pt,wf_gen,wf_r,n_occ_r,coords_r,n_pos_r,inds_A)

    Amp_inds=np.zeros(Lsub+1)
    Phase_inds=np.zeros(Lsub+1,dtype=np.complex64)
    Phase_inds_comp=np.zeros(Lsub+1,dtype=np.complex64)
    
    low = max(int(Lsub/2)-2,0)
    high = min(int(Lsub/2)+3,Lsub+1)
    Na_list=np.arange(low,high)   #Only evaluate withing ~2 of half-filling

    for N_pt_A in Na_list:    
        print("filling of ", N_pt_A)
        for i_r in range(r):
            n_occ_r[:,i_r], n_pos_r[:,i_r], coords_r[:,i_r]=initialize_wf_region(N, N_pt, inds_A, N_pt_A)
            wf_r[:,:,i_r]=np.transpose(wf_gen[coords_r[:,i_r],:])
            cond_r[i_r]=np.linalg.cond(wf_r[:,:,i_r])

        epsilon= 1e-8
        counter=0
        check = np.max(cond_r)
        while check > 1/epsilon :
            counter+=1
            assert counter<=40, "wave function cannot be constructed!"
            for i_r in range(r):
                n_occ_r[:,i_r], n_pos_r[:,i_r], coords_r[:,i_r]=initialize_wf_region(N, N_pt, inds_A, N_pt_A)
                wf_r[:,:,i_r]=np.transpose(wf_gen[coords_r[:,i_r],:])
                cond_r[i_r]=np.linalg.cond(wf_r[:,:,i_r])
                check = np.max(cond_r)

        Phase_inds[N_pt_A]=VMC_phase_ratio(numconfig,check_step,N,N_pt,wf_gen,wf_r,n_occ_r,n_pos_r,coords_r,inds_A,N_pt_A)

        Amp_inds[N_pt_A]=VMC_amplitude_ratio(numconfig,check_step,N,N_pt,wf_gen,wf_r,n_occ_r,n_pos_r,coords_r,inds_A,N_pt_A)



#     ent_vmc= -np.log(np.sum(ent_fr*ent_amp*ent_phase))

#     return ent_vmc
    return Frac, Amp_inds, Phase_inds

In [91]:
N = 20
N_pt = np.int(N/2)

numconfig = 5000
check_step =10
Ns=4                   #maximum length of random walk

Lsub_list= np.arange(1,6,1)

t = 1
dt = 0
BC=-1  #np.exp(1j*np.pi/3)
r=3                             #Degree of Renyi entropy

Orbitals = wf_gen(N,N_pt,BC,t+dt,t-dt)

Rr_ex=np.zeros(len(Lsub_list))
Rr_vmc=np.zeros(len(Lsub_list), dtype=np.complex64)
Gmat=np.dot(Orbitals,np.matrix(Orbitals).H)

##################################################
t_timer=time.time()
random.seed(time.time())

for index in Lsub_list:
    inds_A = np.arange(index)
    Lsub=index
    print('subsystem size = ', Lsub)
    
    Frac, Amp_inds, Phase_inds=Renyi_vmc(numconfig,Orbitals,N, N_pt,Lsub,r)
    
    Rr_ex[index-1]=exact_renyi_calc(r,Gmat[np.ix_(inds_A,inds_A)])
    Rr_vmc[index-1] = -np.abs(np.log(np.sum(Frac*Amp_inds*Phase_inds)))/(1-r)
    
    print("Approximate %d Renyi entropy: "%(r), Rr_vmc[index-1], " vs exact value", Rr_ex[index-1] )
elapsed = time.time() - t_timer
print("Program finished, elapsed time =", elapsed, "sec")    

subsystem size =  1
filling of  0
filling of  1
Approximate 3 Renyi entropy:  (0.70202553+0j)  vs exact value 0.6931471805599453
subsystem size =  2
filling of  0
filling of  1
filling of  2
Approximate 3 Renyi entropy:  (0.5829513+0j)  vs exact value 0.5861313501395304
subsystem size =  3
filling of  0
filling of  1
filling of  2
filling of  3
Approximate 3 Renyi entropy:  (0.8387539+0j)  vs exact value 0.8405435372953755
subsystem size =  4
filling of  0
filling of  1
filling of  2
filling of  3
filling of  4
Approximate 3 Renyi entropy:  (0.73853844+0j)  vs exact value 0.7523360584815246
subsystem size =  5
filling of  0
filling of  1
filling of  2
filling of  3
filling of  4
Approximate 3 Renyi entropy:  (0.9161061+0j)  vs exact value 0.9166185074108373
Program finished, elapsed time = 28348.634788274765 sec
